<a href="https://colab.research.google.com/github/TheClockworkk/ML-Labs-Homeworks/blob/main/lab3_numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import linalg as la

import numba
from datetime import datetime

In [26]:
class Himmelblau():
  @staticmethod
  @numba.njit(fastmath=True)
  def fuction(x):
        return (x[0] * x[0] + x[1] - 11) ** 2 + (x[0] + x[1] * x[1] - 7) ** 2

  @staticmethod
  @numba.njit(fastmath=True)
  def get_bounds():
    return np.array([[-5, -5], [5, 5]])
    
  @staticmethod
  @numba.njit(fastmath=True)
  def get_min():
        return np.array([3., 2., 0.])
  
  @staticmethod
  @numba.njit(fastmath=True)
  def gradient(x):
        return np.array([4 * x[0] * (x[0] * x[0] + x[1] - 11) + 2 * (x[0] + x[1] * x[1] - 7), 4 * x[1] * (x[1] * x[1] + x[0] - 7) + 2 * (x[1] + x[0] * x[0] - 11)])

In [27]:
class Matyas():
  @staticmethod
  @numba.njit(fastmath=True)
  def fuction(x):
        return 0.26*(x[0]*x[0] + x[1]*x[1])-0.48*x[0]*x[1]

  @staticmethod
  @numba.njit(fastmath=True)
  def get_bounds():
    return np.array([[-10, -10], [10, 10]])
    
  @staticmethod
  @numba.njit(fastmath=True)
  def get_min():
        return np.array([0., 0., 0.])
  
  @staticmethod
  @numba.njit(fastmath=True)
  def gradient(x):
        return np.array([0.52*x[0] - 0.48 * x[1], 0.52*x[1] - 0.48 * x[0]])

In [28]:
@numba.njit(fastmath=True)
def my_GD(
    fun: callable, grad: callable, start_params: np.ndarray, glob_min: np.ndarray, 
    max_iter: int = 1000, lr: float = 0.1, delta: float = 0.001
    ) -> np.array:

    """Классический градиентный спуск

    Args:
        fun (callable): Функция искуственного ландшафта
        start_params (np.ndarray): Стартовый набор параметров
        glob_min (np.ndarray): Глобальный минимум рассматриваемой функции
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 1000.
        lr (float, optional): Шаг обучения. Defaults to 0.1.
        delta (float, optional): Радиус сходимости. Defaults to 0.001.

    Returns:
        np.array: История градиентного спуска
    """

    # Рассчитываем начальный набор параметров
    params = start_params.copy()
    path = [np.array([params[0], params[1], fun(params)])]
    
    step = 0
    while (step < max_iter and la.norm(path[-1] - glob_min) > delta):

        # Вычисляем новое значение параметров
        params = params - lr * grad(params)

        # Логируем результат
        path.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return path

In [29]:
# Для Химмельблау
start_time = datetime.now()
path = my_GD(Himmelblau.fuction, Himmelblau.gradient, np.array([5.,5.]), Himmelblau.get_min(), max_iter = 1000, lr = 0.001)
print("Himmelblau default sgd optimized\n", datetime.now() - start_time)

Himmelblau default sgd optimized
 0:00:00.830353


In [30]:
# Для Мак Кормика
start_time = datetime.now()
path = my_GD(Matyas.fuction, Matyas.gradient, np.array([-9.,5.]), Matyas.get_min(), max_iter = 1000, lr = 0.1)
print("Matyas default sgd optimized\n", datetime.now() - start_time)

Matyas default sgd optimized
 0:00:00.838443


In [31]:
@numba.njit(fastmath=True)
def inertial_GD(
    fun: callable, grad: callable, start_params: np.ndarray, glob_min: np.ndarray, 
    max_iter: int = 1000, lr: float = 0.1, delta: float = 0.001, beta: float = 0.5
    ) -> np.array:
    
    """Инертный градиентный спуск

    Args:
        fun (callable): Функция искуственного ландшафта
        start_params (np.ndarray): Начальный набор параметров
        glob_min (np.ndarray): Глобальный минимум рассматриваемой функции
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 1000.
        lr (float, optional): Шаг обучения. Defaults to 0.1.
        delta (float, optional): Радиус сходимости. Defaults to 0.001.
        beta (float, optional): Коэффициент энертности. Defaults to 0.5.

    Returns:
        np.array: История градиентного спуска
    """
    # Рассчитываем начальный набор параметров
    params_prev = start_params.copy()
    params = start_params.copy()
    path = [np.array([params[0], params[1], fun(params)])]
    
    step = 0
    while (step < max_iter and la.norm(path[-1] - glob_min) > delta):

        # Вычисляем новое значение параметров
        params_new = params - lr * grad(params) + beta * (params - params_prev)
        params_prev = params
        params = params_new
        
        # Логируем результат
        path.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return path

In [32]:
# Для Химмельблау
start_time = datetime.now()
path = inertial_GD(Himmelblau.fuction, Himmelblau.gradient, np.array([5.,5.]), Himmelblau.get_min(), max_iter = 1000, lr = 0.001)
print("Himmelblau inertial sgd optimized\n", datetime.now() - start_time)

Himmelblau inertial sgd optimized
 0:00:00.473974


In [33]:
# Для Матьяса
start_time = datetime.now()
path = inertial_GD(Matyas.fuction, Matyas.gradient, np.array([-9.,5.]), Matyas.get_min(), max_iter = 1000, lr = 0.1)
print("Matyas inertial sgd optimized\n", datetime.now() - start_time)

Matyas inertial sgd optimized
 0:00:00.488622


In [34]:
@numba.njit(fastmath=True)
def Adam_GD(
    fun: callable, grad: callable, start_params: np.ndarray, glob_min: np.ndarray, max_iter: int = 1000, 
    lr: float = 0.1, b1: float = 0.6, b2: float = 0.999, e: float = 10e-8, delta: float = 0.001
    ) -> np.array:
    
    """Адаптивный градиентный спуск: Adam

    Args:
        fun (callable): Функция искуственного ландшавта
        start_params (np.ndarray): Стартовый набор параметров
        glob_min (np.ndarray): Глобальный минимум рассматриваемой функции
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 1000.
        lr (float, optional): Шаг обучения. Defaults to 0.1.
        b1 (float, optional): Параметр beta1. Defaults to 0.6.
        b2 (float, optional): Параметр beta2. Defaults to 0.999.
        e (float, optional): "Бесконечно малое" число. Defaults to 10e-8.
        delta (float, optional): Радиус сходимости. Defaults to 0.001.

    Returns:
        np.array: История градиентного спуска
    """

    # Рассчитываем начальный набор параметров
    params = start_params.copy()
    path = [np.array([params[0], params[1], fun(params)])]

    # Инициализируем первый и второй моменты
    m = np.array([0., 0.])
    v = np.array([0., 0.])
    
    step = 0
    while (step < max_iter and la.norm(path[-1] - glob_min) > delta):
        
        # Считаем первый первый и второй моменты
        m = b1 * m + (1 - b1) * grad(params)
        v = b2 * v + (1 - b2) * grad(params) ** 2

        # Вычисляем новое значение параметров
        params = params - lr * m / (np.sqrt(v) + e)
        
        # Логируем результат
        path.append(np.array([params[0], params[1], fun(params)]))
        step += 1

    return path

In [35]:
# Для Химмельблау
start_time = datetime.now()
path = Adam_GD(Himmelblau.fuction, Himmelblau.gradient, np.array([5.,5.]), Himmelblau.get_min(), max_iter = 1000, lr = 0.1)
print("Himmelblau Adam sgd optimized\n", datetime.now() - start_time)

Himmelblau Adam sgd optimized
 0:00:00.647958


In [36]:
# Для Матьяса
start_time = datetime.now()
path = Adam_GD(Matyas.fuction, Matyas.gradient, np.array([-9.,5.]), Matyas.get_min(), max_iter = 1000, lr = 0.1)
print("Matyas Adam sgd optimized\n", datetime.now() - start_time)

Matyas Adam sgd optimized
 0:00:00.753242


Himmelblau default sgd optimized
 0:00:00.602195

Himmelblau inertial sgd optimized
 0:00:00.525529

 Himmelblau Adam sgd optimized
 0:00:00.506014

Himmelblau default sgd 0:00:00.602195

Himmelblau inertial sgd 0:00:00.525529

Himmelblau Adam sgd 0:00:00.506014

Matyas default sgd optimized
 0:00:00.542960

 Matyas inertial sgd optimized
 0:00:00.374963
 
 Matyas Adam sgd optimized
 0:00:00.626916

Matyas default sgd 0:00:00.542960

Matyas inertial sgd 0:00:00.374963

Matyas Adam sgd 0:00:00.626916